In [127]:
# autor: Kamil Tasarz

In [128]:
using Printf

In [129]:
#przygotowanie

two_to = (2.0 .^ (1:62)) #tablica kolejnych potęg dwójki
four_to = (4.0 .^ (1:31)) #tablica kolejnych potęg czwórki

31-element Vector{Float64}:
      4.0
     16.0
     64.0
    256.0
   1024.0
   4096.0
  16384.0
  65536.0
 262144.0
      1.048576e6
      4.194304e6
      1.6777216e7
      6.7108864e7
      ⋮
      1.099511627776e12
      4.398046511104e12
      1.7592186044416e13
      7.0368744177664e13
      2.81474976710656e14
      1.125899906842624e15
      4.503599627370496e15
      1.8014398509481984e16
      7.205759403792794e16
      2.8823037615171174e17
      1.152921504606847e18
      4.611686018427388e18

In [130]:
function trapez(f, a, b, n, h)
    res = 0.0
    for i in 0:n
        if i==0 || i==n
            res += f(a + i*h) / 2.0
        else
            res += f(a + i*h)
        end
    end
    return res*h
end

function summa(to, f, a, h) # aktulanie nieużywana, można użyć zamiast trapez
    s = 0.0
    for i in 1:to
        s = s + f(a + (2.0 * i - 1.0) * h)
    end
    return s
end

summa (generic function with 2 methods)

In [150]:
# R_(n, m), gdzie n - wiersz, m - kolumna
# przy czym Julia numeruje tablice od 1 a nie od zera, więc wyraz R_(i, j) jest w miejscu M[i+1][j+1]

function Romberg_KT(a, b, f, ϵ, drukowac=false, exact=0.0)
    # początek i koniec przedzialu całki, funkcja, oczekiwany błąd, czy wypisać tablicę błędów oraz
    # dokładna wartość całki (potrzebna do wypisania tablicy błędów; jeśli nie podamy otrzymamy po prostu tablicę Romberga)
    # zwracana wynik to liczba wierszy i najlepsze uzyskane przybliżenie całki
    M = [] #tablica Romberga
    h = (b - a)
    R00 = h * (f(b)-f(a)) / 2.0
    push!(M, R00)
    i = 1 #numer wiersza
    h = (b - a)
    
    while true
        w = [] #aktualny wiersz
        h = h / 2.0
        # R = M[i][1]/2.0 + h*summa(1, two_to[i]/2, f, a, h) #nieco szybsze
        R = trapez(f, a, b, two_to[i], h)
        push!(w, R)
        
        for j in 1:1:i
           # R = w[j] + (w[j] - M[i][j])/(four_to[j] - 1.0)
           R = (four_to[j] * w[j] - M[i][j]) / (four_to[j] - 1.0) # drugi równoważny sposób
            push!(w, R)
        end
        push!(M, w)
        
        if(abs(M[i+1][1] - M[i][1]) < ϵ*abs(M[i+1][1])) #warunek z treści zadania: |R_(K, 0) - R_(K-1, 0)| < ϵ*|R_(K, 0)|
            break
        end
        
        i+=1
    end
            
    if drukowac
        #drukuj_ostatnie_wyrazy_tablicy(M)
        drukuj_tablice_bledow(M, exact)
    end
    
    return i, M[i+1][i+1]
end

     

Romberg_KT (generic function with 3 methods)

In [132]:
function drukuj_tablice_bledow(M, exact)
    if exact == 0.0 # drukowanie po prostu tablicy Romberga
        for i in 1:1:length(M)
            @printf("%d: ", i-1)
            for j in 1:1:i
                @printf("%.5f ", M[i][j])
            end
            @printf("\n")
        end
    else #drukowanie błędów
        for i in 1:1:length(M)
            @printf("%d: ", i-1)
            for j in 1:1:i
                @printf("%.3e ", abs(exact - M[i][j]) / abs(exact))
            end
            @printf("\n")
        end
    end  
end

function drukuj_ostatnie_wyrazy_tablicy(M) # zamiast wypisywać całe wiersze można wypisać najlepsze przybliżenie z każdego
    for i in 1:1:length(M)
            @printf("%d: %f \n", i-1, M[i][i])
        end
end

drukuj_ostatnie_wyrazy_tablicy (generic function with 1 method)

In [134]:
#test1 - wielomian
a = -1.0
b = 1.0
f(x) = 2.0 * x^3 - 6.0/5.0 * x^2 + 0.17
exact = -0.46

print("epsilon i wynik funckji Romberg", "\n")
for ϵ in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    @printf("%.1e  ", ϵ)
    print(Romberg_KT(a, b, f, ϵ, false, exact), "\n")
end

epsilon i wynik funckji Romberg
1.0e-01  (4, -0.4599916173876962)
1.0e-02  (6, -0.4599999999979983)
1.0e-03  (7, -0.4600000000000031)
1.0e-04  (9, -0.46000000000000396)
1.0e-05  (10, -0.45999999999995933)
1.0e-06  (12, -0.46000000000004754)
1.0e-07  (14, -0.4599999999997553)
1.0e-08  (15, -0.459999999999876)


In [144]:
#test2 - funkcja wykładnicza
a = -1.0
b = 1.0
f(x) = ℯ^x
exact = 2.350402387287602913

print("epsilon i wynik funckji Romberg", "\n")
for ϵ in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    @printf("%.1e  ", ϵ)
    print(Romberg_KT(a, b, f, ϵ, false, exact), "\n")
end

epsilon i wynik funckji Romberg
1.0e-01  (1, 2.6073067173244575)
1.0e-02  (4, 2.3504013695769763)
1.0e-03  (5, 2.350402388282478)
1.0e-04  (7, 2.3504023872876028)
1.0e-05  (9, 2.350402387287601)
1.0e-06  (10, 2.3504023872876045)
1.0e-07  (12, 2.3504023872876045)
1.0e-08  (14, 2.3504023872876063)


In [136]:
#test3 - funkcja Rungego
a = -1.0
b = 1.0
f(x) = 1.0 / (1.0 + 25.0 * x^2)
exact = 0.549360306778006344344508770577

print("epsilon i wynik funckji Romberg", "\n")
for ϵ in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    @printf("%.1e  ", ϵ)
    print(Romberg_KT(a, b, f, ϵ, false, exact), "\n")
end

epsilon i wynik funckji Romberg
1.0e-01  (4, 0.5487063518598903)
1.0e-02  (5, 0.5495459860498154)
1.0e-03  (5, 0.5495459860498154)
1.0e-04  (6, 0.549358036735076)
1.0e-05  (8, 0.5493603068692027)
1.0e-06  (10, 0.5493603067780057)
1.0e-07  (11, 0.5493603067780068)
1.0e-08  (13, 0.5493603067780053)


In [137]:
#test4 - z treści zadania
a = -1.0
b = 1.0
f(x) = 1.0 / (x^4 + x^2 + 0.9)
exact = 1.58223

print("epsilon i wynik funckji Romberg", "\n")
for ϵ in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    @printf("%.1e  ", ϵ)
    print(Romberg_KT(a, b, f, ϵ, false, exact), "\n")
end

epsilon i wynik funckji Romberg
1.0e-01  (2, 1.5612171004990063)
1.0e-02  (4, 1.5822386398082946)
1.0e-03  (5, 1.5822329223318037)
1.0e-04  (7, 1.5822329637296086)
1.0e-05  (9, 1.5822329637296728)
1.0e-06  (10, 1.5822329637296693)
1.0e-07  (12, 1.582232963729682)
1.0e-08  (14, 1.5822329637296866)


In [138]:
#test5 - z treści zadania
a = 0.0
b = 1.0
f(x) = 1.0 / (1.0 + x^4)
exact = 0.866972987339911037573995163882870713652175367

print("epsilon i wynik funckji Romberg", "\n")
for ϵ in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    @printf("%.1e  ", ϵ)
    print(Romberg_KT(a, b, f, ϵ, false, exact), "\n")
end

epsilon i wynik funckji Romberg
1.0e-01  (2, 0.8442023263292009)
1.0e-02  (3, 0.8673336322080251)
1.0e-03  (5, 0.8669729885775816)
1.0e-04  (6, 0.8669729873397677)
1.0e-05  (8, 0.8669729873399116)
1.0e-06  (10, 0.8669729873399111)
1.0e-07  (11, 0.8669729873399112)
1.0e-08  (13, 0.866972987339908)


In [145]:
#test6 - z treści zadania
a = 0.0
b = 1.0
f(x) = 2.0 / (2.0 + sin(10*pi*x))
ϵ = 1e-8
exact = 1.1547005383792515290182975610039149 # 2/sqrt(3)

print("epsilon i wynik funckji Romberg", "\n")
for ϵ in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    @printf("%.1e  ", ϵ)
    print(Romberg_KT(a, b, f, ϵ, false, exact), "\n")
end

epsilon i wynik funckji Romberg
1.0e-01  (3, 1.1449399512891578)
1.0e-02  (4, 1.15512033569102)
1.0e-03  (4, 1.15512033569102)
1.0e-04  (4, 1.15512033569102)
1.0e-05  (5, 1.1546962210990694)
1.0e-06  (5, 1.1546962210990694)
1.0e-07  (5, 1.1546962210990694)
1.0e-08  (5, 1.1546962210990694)


In [143]:
#test drukowania tablicy błędów
a = -1.0
b = 1.0
ϵ = 1e-6
f(x) = sqrt(sqrt(sqrt(1.0 - x))) # (1-x)^(1/8)
exact = 1.9386804136271247274791300546857030

Romberg_KT(a, b, f, ϵ, true, exact)

0: 1.562e+00 
1: 2.029e-01 2.503e-01 
2: 9.365e-02 5.722e-02 7.772e-02 
3: 4.309e-02 2.624e-02 2.417e-02 2.332e-02 
4: 1.980e-02 1.203e-02 1.108e-02 1.088e-02 1.083e-02 
5: 9.086e-03 5.516e-03 5.082e-03 4.986e-03 4.963e-03 4.958e-03 
6: 4.168e-03 2.529e-03 2.330e-03 2.286e-03 2.276e-03 2.273e-03 2.272e-03 
7: 1.912e-03 1.160e-03 1.068e-03 1.048e-03 1.043e-03 1.042e-03 1.042e-03 1.042e-03 
8: 8.767e-04 5.317e-04 4.898e-04 4.806e-04 4.784e-04 4.778e-04 4.777e-04 4.777e-04 4.777e-04 
9: 4.020e-04 2.438e-04 2.246e-04 2.204e-04 2.193e-04 2.191e-04 2.190e-04 2.190e-04 2.190e-04 2.190e-04 
10: 1.843e-04 1.118e-04 1.030e-04 1.010e-04 1.006e-04 1.005e-04 1.004e-04 1.004e-04 1.004e-04 1.004e-04 1.004e-04 
11: 8.452e-05 5.125e-05 4.721e-05 4.633e-05 4.611e-05 4.606e-05 4.605e-05 4.604e-05 4.604e-05 4.604e-05 4.604e-05 4.604e-05 
12: 3.875e-05 2.350e-05 2.165e-05 2.124e-05 2.114e-05 2.112e-05 2.111e-05 2.111e-05 2.111e-05 2.111e-05 2.111e-05 2.111e-05 2.111e-05 
13: 1.777e-05 1.077e-05 9.925e-06 9

(17, 1.938679584353254)